In [ ]:
!pip install --upgrade -r requirements.txt

In [1]:
!TOKENIZERS_PARALLELISM=true wget -O biblioteca.pdf "https://anexosportal.datalegis.net/arquivos/1878261.pdf"

'TOKENIZERS_PARALLELISM' nÆo ‚ reconhecido como um comando interno
ou externo, um programa oper vel ou um arquivo em lotes.


## Código para popular um Banco de Dados Vetorial

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

dados = PyPDFLoader("Dados/D11246.pdf").load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
textos = text_splitter.split_documents(dados)
embedding_engine = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
vector_db = Chroma.from_documents(textos, embedding_engine)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\jeferson.oliveira\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jeferson.oliveira\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Código para montar um RAG

In [7]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
n_documentos = 1

def format_docs(documentos):
    return "\n\n".join(documento.page_content for documento in documentos)

llm = ChatOpenAI(openai_api_key = OPENAI_API_KEY, model = "gpt-4o-mini")
prompt = hub.pull("rlm/rag-prompt")

rag = (
    {
        "question": RunnablePassthrough(),
        "context": vector_db.as_retriever(k = n_documentos) 
                    | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)

C:\Users\jeferson.oliveira\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## Código para perguntar ao RAG

In [9]:
prompt = "Qual é a função do fiscal técnico?"
rag.invoke(prompt)

'A função do fiscal técnico é assegurar que as exigências de caráter técnico do contrato sejam cumpridas, incluindo a fiscalização e a avaliação das obrigações assumidas pelo contratado. Ele também auxilia o gestor do contrato com informações necessárias e realiza o recebimento provisório do objeto do contrato. Em caso de afastamento, um substituto deve assumir suas responsabilidades.'